# Helper

In [ ]:
import cv2
import numpy as np
import os

def load_data(base_path, batch_size=8):
    # Iterate over the three train directories (train/1, train/2, train/3)
    for fragment_id in range(1, 4):
        fragment_path = os.path.join(base_path, str(fragment_id))
        print(f"Loading data from fragment: {fragment_id}")

        images, masks, inklabels = [], [], []
        for i in range(65):  # Load surface_volume images (00.tif to 64.tif)
            image_path = os.path.join(fragment_path, f'surface_volume/{i:02d}.tif')
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            images.append(image)

            # Load mask.png and inklabels.png only once per fragment
            if i == 0:
                mask = cv2.imread(os.path.join(fragment_path, 'mask.png'), cv2.IMREAD_GRAYSCALE)
                inklabel = cv2.imread(os.path.join(fragment_path, 'inklabels.png'), cv2.IMREAD_GRAYSCALE)
            
            # Append the same mask and inklabel for each image slice
            masks.append(mask)
            inklabels.append(inklabel)

            # Yield a batch of data
            if len(images) == batch_size:
                yield np.stack(images, axis=0), np.stack(masks, axis=0), np.stack(inklabels, axis=0)
                images, masks, inklabels = [], [], []

        # Yield any remaining data
        if images:
            yield np.stack(images, axis=0), np.stack(masks, axis=0), np.stack(inklabels, axis=0)

    print("Data loading complete.")

# Usage example:
base_path = '/kaggle/input/vesuvius-challenge-ink-detection/train'
for images, masks, inklabels in load_data(base_path, batch_size=8):
    # Process the batch of data
    pass

Loading data from fragment: 1
Loading data from fragment: 2


# Dataset

In [ ]:
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import os
from torch.utils.data import Dataset

class VesuviusDataset(Dataset):
    def __init__(self, base_path, transforms=None):
        self.base_path = base_path
        self.transforms = transforms
        self.fragment_ids = [str(i) for i in range(1, 4)]
        self.slice_ids = [f'{i:02d}' for i in range(65)]

    def __len__(self):
        return len(self.fragment_ids) * len(self.slice_ids)

    def __getitem__(self, idx):
        fragment_idx = idx // len(self.slice_ids)
        slice_idx = idx % len(self.slice_ids)
        fragment_id = self.fragment_ids[fragment_idx]
        slice_id = self.slice_ids[slice_idx]

        fragment_path = os.path.join(self.base_path, fragment_id)
        image = cv2.imread(os.path.join(fragment_path, f'surface_volume/{slice_id}.tif'), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(os.path.join(fragment_path, 'mask.png'), cv2.IMREAD_GRAYSCALE)
        inklabel = cv2.imread(os.path.join(fragment_path, 'inklabels.png'), cv2.IMREAD_GRAYSCALE)

        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask, inklabel

def get_transforms():
    # Define data augmentation and preprocessing transforms
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(mean=[0.485], std=[0.229]),  # Updated mean and std for grayscale images
        ToTensorV2(),
    ], p=1.0)

# Usage example:
base_path = '/kaggle/input/vesuvius-challenge-ink-detection/train'
transforms = get_transforms()  # Define your data augmentation and preprocessing transforms
dataset = VesuviusDataset(base_path, transforms=transforms)

# Access data using indices
image, mask, inklabel = dataset[0]  # Get the first item from the dataset

# Model

In [ ]:
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
import sys
sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels')
sys.path.append('/kaggle/input/segmentation-models-pytorch/segmentation_models.pytorch-master')
sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch-master/efficientnet_pytorch')

import pretrainedmodels
import segmentation_models_pytorch as smp
import efficientnet_pytorch


NUM_FOLDS = 5
NUM_EPOCHS = 30
BATCH_SIZE = 8
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BACKBONE = 'efficientnet-b4'
LEARNING_RATE = 1e-4

def get_model():
    model = smp.DeepLabV3Plus(
        encoder_name=BACKBONE,
        encoder_weights='imagenet',
        in_channels=1,
        classes=1,
    )
    # Parallelize the model across multiple GPUs
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    return model.to(DEVICE)
    
def train_one_fold(fold, train_loader, val_loader):
    print(f"Training fold {fold}")
    model = get_model().to(DEVICE)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = GradScaler()  # For mixed precision training

    for epoch in range(NUM_EPOCHS):
        print(f"Starting epoch {epoch}")
        # Training loop
        model.train()
        for batch in train_loader:
            images, masks = batch
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            optimizer.zero_grad()
            with autocast():  # Mixed precision training
                outputs = model(images)
                loss = criterion(outputs, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                images, masks = batch
                images, masks = images.to(DEVICE), masks.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Fold {fold}, Epoch {epoch}, Validation Loss: {val_loss}")

    return model

def kfold_cross_validation():
    # Load and preprocess data
    images, masks = load_data('/kaggle/input/vesuvius-challenge-ink-detection/train')  # Replace with the correct path to your data
    
    # Flatten the lists of fragments into a single list of image slices and masks
    images = [img_slice for fragment in images for img_slice in fragment]
    masks = [mask_slice for fragment_masks in masks for mask_slice in fragment_masks if fragment_masks is not None]
    
    dataset = VesuviusDataset(images, masks, transforms=get_transforms())
    
    kfold = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    models = []
    
    for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset)):
        train_subset = torch.utils.data.Subset(dataset, train_indices)
        val_subset = torch.utils.data.Subset(dataset, val_indices)
        
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=8, shuffle=False, num_workers=4)
        
        model = train_one_fold(fold, train_loader, val_loader)
        models.append(model)
    
    return models

def ensemble_models(models):
    print("Ensembling models")
    def ensemble(images):
        with torch.no_grad():
            preds = [model(images) for model in models]
            avg_preds = sum(preds) / len(preds)
        return avg_preds

    return ensemble

# Main

In [ ]:
def kfold_cross_validation():
    # Load and preprocess data
    train_base_path = '/kaggle/input/vesuvius-challenge-ink-detection/train'
    
    # Create lists to store all images, masks, and inklabels
    all_images, all_masks, all_inklabels = [], [], []
    
    # Iterate over the generator to get all batches of data
    for images, masks, inklabels in load_data(train_base_path):
        all_images.extend(images)
        all_masks.extend(masks)
        all_inklabels.extend(inklabels)
    
    # Convert lists to numpy arrays
    all_images = np.stack(all_images, axis=0)
    all_masks = np.stack(all_masks, axis=0)
    all_inklabels = np.stack(all_inklabels, axis=0)
    
    dataset = VesuviusDataset(all_images, all_masks, transforms=get_transforms())
    
    kfold = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    models = []
    
    for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset)):
        train_subset = torch.utils.data.Subset(dataset, train_indices)
        val_subset = torch.utils.data.Subset(dataset, val_indices)
        
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=8, shuffle=False, num_workers=4)
        
        model = train_one_fold(fold, train_loader, val_loader)
        models.append(model)
    
    return models

# K-Fold Cross Validation and training
models = kfold_cross_validation()

# Ensembling models
ensemble_fn = ensemble_models(models)

# Load test data
test_base_path = '/kaggle/input/vesuvius-challenge-ink-detection/test'
test_images = []
for fragment_id in range(1, 4):
    fragment_path = os.path.join(test_base_path, str(fragment_id))
    for i in range(65):
        image_path = os.path.join(fragment_path, f'surface_volume/{i:02d}.tif')
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        test_images.append(image)

test_data_loader = DataLoader(VesuviusDataset(test_images, None, get_transforms()), batch_size=BATCH_SIZE, shuffle=False)

# Predict and create submission file
predictions = predict(ensemble_fn, test_data_loader)
create_submission_file(predictions)

# Submission

In [ ]:
import pandas as pd

def predict(ensemble_fn, test_data_loader):
    predictions = []
    with torch.no_grad():
        for batch in test_data_loader:
            images = batch
            images = images.to(DEVICE)
            outputs = ensemble_fn(images)
            predictions.extend(outputs.cpu().numpy())
    return predictions

def create_submission_file(predictions):
    submission = pd.DataFrame(columns=['Id', 'Predicted'])
    for i, pred in enumerate(predictions):
        rle = rle_encoding(pred.squeeze())
        submission = submission.append({'Id': str(i), 'Predicted': ' '.join(map(str, rle))}, ignore_index=True)
    submission.to_csv('submission.csv', index=False)

# Load and preprocess data
images, masks = load_data('train')  # Replace with the correct path to your data

# K-Fold Cross Validation and training
models = kfold_cross_validation()

# Ensembling models
ensemble_fn = ensemble_models(models)

# Load test data
test_images = [fragment for fragment_id in os.listdir('test') for fragment in load_data(os.path.join('test', fragment_id))[0]]  # Replace with the correct path to your test data

# Flatten the list of test fragments into a single list of image slices
test_images = [img_slice for fragment in test_images for img_slice in fragment]

test_data_loader = DataLoader(VesuviusDataset(test_images, None, get_transforms()), batch_size=BATCH_SIZE, shuffle=False)

# Predict and create submission file
predictions = predict(ensemble_fn, test_data_loader)
create_submission_file(predictions)